In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore

benin_data = pd.read_csv('../data/benin-malanville.csv')
sierraleone_data = pd.read_csv('../data/sierraleone-bumbuna.csv')
togo_data = pd.read_csv('../data/togo-dapaong_qc.csv')

# Function to summarize data using pandas' describe() method, which returns summary statistics for all numeric columns by default.
def summarize_data(df):
    return df.describe(include='all')  # 'include=all' to get statistics for all columns

benin_summary = summarize_data(benin_data)
sierraleone_summary = summarize_data(sierraleone_data)
togo_summary = summarize_data(togo_data)

print("Benin Summary Statistics:\n", benin_summary)
print("Sierra Leone Summary Statistics:\n", sierraleone_summary)
print("Togo Summary Statistics:\n", togo_summary)

# Function for data quality check using pandas isnull() and select_dtypes() method.
def data_quality_check(df):
    missing_values = df.isnull().sum()
    negative_values = df[df.select_dtypes(include='number') < 0].count()
    return missing_values, negative_values

benin_missing, benin_negatives = data_quality_check(benin_data)
sierraleone_missing, sierraleone_negatives = data_quality_check(sierraleone_data)
togo_missing, togo_negatives = data_quality_check(togo_data)

print("Benin Data Quality Check:\n", benin_missing, benin_negatives)
print("Sierra Leone Data Quality Check:\n", sierraleone_missing, sierraleone_negatives)
print("Togo Data Quality Check:\n", togo_missing, togo_negatives)


# Function for Plot line graphs or area plots to observe patterns over time (modified code to include multiple parameters).


def plot_time_series_all(df_dict, columns, titles):
    for country, df in df_dict.items():
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
        for column, title in zip(columns, titles):
            if column in df.columns:
                plt.figure(figsize=(12, 6))
                plt.plot(df['Timestamp'], df[column], label=column)
                plt.title(f'{country} - {title}')
                plt.xlabel('Time')
                plt.ylabel(column)
                plt.legend()
                plt.show()
            else:
                print(f"Column '{column}' not found in {country} data.")

# DataFrames for each country
df_dict = {
    'Benin': benin_data,
    'Sierra Leone': sierraleone_data,
    'Togo': togo_data
}

# Columns to plot
columns = [
    'GHI', 'DNI', 'DHI', 'Tamb'
]

# Titles for each column
titles = [
    'Global Horizontal Irradiance Over Time',
    'Direct Normal Irradiance Over Time',
    'Diffuse Horizontal Irradiance Over Time',
    'Ambient Temperature Over Time',
 
]
# we can decrease the number of columns as needed, the above code is to show how flexible my code is to inlude all columns at the same time
# Plot time series for each column and country
plot_time_series_all(df_dict, columns, titles)



# Impact (how cleaning impacts sensor readings over time.)

def cleaning_impact_plot(df, column):
    plt.figure(figsize=(12, 6))
    plt.scatter(df[df['Cleaning'] == 1]['Timestamp'], df[df['Cleaning'] == 1][column], color='blue', label='Cleaning')
    plt.scatter(df[df['Cleaning'] == 0]['Timestamp'], df[df['Cleaning'] == 0][column], color='red', label='No Cleaning')
    plt.title(f'Impact of Cleaning on {column}')
    plt.xlabel('Time')
    plt.ylabel(column)
    plt.legend()
    plt.show()

# Plot for each country
cleaning_impact_plot(benin_data, 'ModA')
cleaning_impact_plot(togo_data, 'ModA')


# For analyzing correlation anaylysic of all columns ecept non numberic values
def correlation_analysis(df, title):
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    
    plt.figure(figsize=(12, 8))
    correlation_matrix = numeric_df.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title(f'Correlation Matrix - {title}')
    plt.show()


# Correlation analysis for each country
correlation_analysis(benin_data, 'Benin')
correlation_analysis(sierraleone_data, 'Sierra Leone')
correlation_analysis(togo_data, 'Togo')

# For analyzing wind speed and direction, a polar chart for wind direction and speed, or a time series plot
def wind_analysis(df, country):
    plt.figure(figsize=(12, 6))
    plt.scatter(df['WD'], df['WS'], alpha=0.5)
    plt.title(f'{country} - Wind Speed vs Wind Direction')
    plt.xlabel('Wind Direction (degrees)')
    plt.ylabel('Wind Speed (m/s)')
    plt.show()

# Wind analysis for each country
wind_analysis(benin_data, 'Benin')
wind_analysis(sierraleone_data, 'Sierra Leone')
wind_analysis(togo_data, 'Togo')

# For analyzing temperature, or a time series plot
def temperature_analysis(df, country):
    plt.figure(figsize=(12, 6))
    plt.plot(df['Timestamp'], df['Tamb'], label='Ambient Temperature')
    plt.plot(df['Timestamp'], df['TModA'], label='Module A Temperature')
    plt.plot(df['Timestamp'], df['TModB'], label='Module B Temperature')
    plt.title(f'{country} - Temperature Over Time')
    plt.xlabel('Time')
    plt.ylabel('Temperature (°C)')
    plt.legend()
    plt.show()

# Temperature analysis for each country
temperature_analysis(benin_data, 'Benin')
temperature_analysis(sierraleone_data, 'Sierra Leone')
temperature_analysis(togo_data, 'Togo')


# Histograms for overview of the distribution of various sensor readings.
def plot_histograms(df, columns, country):
    df[columns].hist(figsize=(15, 10), bins=20)
    plt.suptitle(f'{country} - Histograms of Sensor Readings')
    plt.show()

# Plot histograms for each country
columns_to_plot = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'Tamb']
plot_histograms(benin_data, columns_to_plot, 'Benin')
plot_histograms(sierraleone_data, columns_to_plot, 'Sierra Leone')
plot_histograms(togo_data, columns_to_plot, 'Togo')


# Z-Score to identify anomalies in the given data, such as outliers in sensor reading
def z_score_analysis(df, column, country):
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')    
    # Calculate Z-scores
    df['zscore'] = zscore(df[column].dropna())  # Ensure column is numeric
    
    # Plot Z-scores
    plt.figure(figsize=(12, 6))
    plt.plot(df['Timestamp'], df['zscore'])
    plt.axhline(y=3, color='r', linestyle='--', label='Threshold')
    plt.title(f'{country} - Z-Score Analysis for {column}')
    plt.xlabel('Time')
    plt.ylabel('Z-Score')
    plt.legend()
    plt.show()


# Z-Score analysis for each country
z_score_analysis(benin_data, 'ModA', 'Benin')
z_score_analysis(sierraleone_data, 'ModA', 'Sierra Leone')
z_score_analysis(togo_data, 'ModA', 'Togo')

# Bubble plot for each country
def bubble_chart(df, x_col, y_col, size_col, color_col, country):
    plt.figure(figsize=(12, 6))
    plt.scatter(df[x_col], df[y_col], s=df[size_col]*10, c=df[color_col], alpha=0.5, cmap='viridis')
    plt.colorbar(label=color_col)
    plt.title(f'{country} - Bubble Chart: {x_col} vs {y_col}')
    plt.xlabel(x_col)
    plt.ylabel(y_col)
    plt.show()

# Bubble chart for each country
bubble_chart(benin_data, 'DNI', 'ModA', 'WS', 'Tamb', 'Benin')
bubble_chart(sierraleone_data, 'DNI', 'ModA', 'WS', 'Tamb', 'Sierra Leone')
bubble_chart(togo_data, 'DNI', 'ModA', 'WS', 'Tamb', 'Togo')


# Clean data 
def clean_data(df):
    # Step 1: Drop columns that are entirely null
    df = df.dropna(axis=1, how='all').copy()  # Make a copy to avoid modifying the original DataFrame

    # Step 2: Fill missing values
    for column in df.columns:
        if df[column].dtype == 'object':  # Handle non-numeric columns
            df.loc[:, column] = df[column].fillna('Unknown')
        else:  # Forward fill for numeric columns
            df.loc[:, column] = df[column].ffill()

    # Step 3: Remove outliers using Z-Score
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df = df[(zscore(df[numeric_cols]) < 3).all(axis=1)].copy()

    # Step 4: Drop duplicate rows
    df = df.drop_duplicates().copy()

    # Step 5: Convert 'Timestamp' column to datetime if it exists
    if 'Timestamp' in df.columns:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')

    return df

# Clean data for each country
benin_data_cleaned = clean_data(benin_data)
sierraleone_data_cleaned = clean_data(sierraleone_data)
togo_data_cleaned = clean_data(togo_data)

# Print the cleaned data summaries to verify the results
print("Benin Data After Cleaning:\n", benin_data_cleaned.head())  # Printing the first few rows
print("Sierra Leone Data After Cleaning:\n", sierraleone_data_cleaned.head())
print("Togo Data After Cleaning:\n", togo_data_cleaned.head())